In [1]:
import os                  # for usernanme y set direcotrio
import pandas as pd
import numpy as np
import weightedcalcs as wc # ponderador o factor de expansion
from tqdm import tqdm      # controlar el tiempo en un loop ()
import warnings
warnings.filterwarnings('ignore')

## Merge Dataset (Python y R)
---

#### 2019

In [2]:
# Cargando bases de datos 2019
enaho01 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2019/687-Modulo01/687-Modulo01/enaho01-2019-100.dta")
enaho34 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2019/687-Modulo34/687-Modulo34/sumaria-2019.dta")

In [3]:
# identificador por hogar: conglome, vivienda, hogar

merge_base_2019 = pd.merge(enaho01, enaho34,
                       on = ["conglome", "vivienda", "hogar"],
                       how = "left",
                       validate = "m:1",
                       suffixes=('', '_y'))

merge_base_2019.shape

(43868, 478)

In [4]:
## drop variables que tenrminan en _y
index_columns = np.where( merge_base_2019.columns.str.contains('_y$', regex=True))[0]
merge_base_2019.drop(merge_base_2019.columns[index_columns], axis = 1, inplace = True)

merge_base_2019.shape

(43868, 471)

#### 2020

In [5]:
# Cargando bases de datos 2020
enaho01 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo01/737-Modulo01/enaho01-2020-100.dta")
enaho34 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo34/737-Modulo34/sumaria-2020.dta")


In [6]:
# identificador por hogar: conglome, vivienda, hogar

merge_base_2020 = pd.merge(enaho01, enaho34,
                       on = ["conglome", "vivienda", "hogar"],
                       how = "left",
                       validate = "m:1",
                       suffixes=('', '_y'))

merge_base_2020.shape

(53423, 507)

In [7]:
## drop variables que tenrminan en _y
index_columns = np.where( merge_base_2020.columns.str.contains('_y$', regex=True))[0]
merge_base_2020.drop(merge_base_2020.columns[index_columns], axis = 1, inplace = True)

merge_base_2020.shape

(53423, 497)

#### juntando bases 2019 y 2020

In [8]:
# Juntando bases del 2019 y 2020
base_final = merge_base_2019.append(merge_base_2020, ignore_index= True)  # ignore_index=True, para evitar conflictos de index
base_final

,aÑo,mes,nconglome,conglome,vivienda,hogar,ubigeo,dominio,estrato,periodo,...,ingtpu071,ingtpu08,ingtpu081,ingtpu09,ingtpu091,inghog1d1,inghog2d1,lineav_rpl,lineav,pobrezav
0,2019,10,007060,005001,007,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,10,007060,005001,017,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,10,007060,005001,028,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,10,007060,005001,040,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,10,007060,005001,050,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97286,2020,12,015122,020305,052,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,0.0,0.0,0.0,0.0,0.0,11442.000000,11442.000000,657.0,642.814026,no vulnerable
97287,2020,12,015122,020305,073,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,0.0,0.0,0.0,0.0,0.0,12083.000000,12083.000000,657.0,641.782898,no vulnerable
97288,2020,12,015122,020305,094,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97289,2020,12,015122,020305,116,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,0.0,0.0,0.0,0.0,0.0,21009.353516,21009.353516,657.0,643.160034,vulnerable no pobre


In [9]:
# extrae 2 primeros digitos

base_final['ubigeo_dep'] = base_final['ubigeo'].str[:2]

In [10]:
# convirtiendo variables a int para futuro merge

base_final[['aÑo', 'ubigeo_dep']] = base_final[['aÑo', 'ubigeo_dep']].astype(int)

#### Deflactor temporal

In [11]:
### Deflactor temporal
deflactores_base2020_new = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo34/737-Modulo34/ConstVarGasto-Metodologia actualizada/Gasto2020/Bases/deflactores_base2020_new.dta")
deflactores_base2020_new = deflactores_base2020_new[ ["dpto", "aniorec", "i00"] ]
deflactores_base2020_new

,dpto,aniorec,i00
0,1.0,2001.0,0.607886
1,2.0,2001.0,0.565700
2,3.0,2001.0,0.636805
3,4.0,2001.0,0.532813
4,5.0,2001.0,0.571589
...,...,...,...
475,21.0,2020.0,1.000000
476,22.0,2020.0,1.000000
477,23.0,2020.0,1.000000
478,24.0,2020.0,1.000000


In [12]:
# identificador por hogar: conglome, vivienda, hogar

base_final = pd.merge(base_final, deflactores_base2020_new,
                      left_on = ["ubigeo_dep", "aÑo"],
                      right_on = ["dpto", "aniorec"],
                      how = "left",
                      validate = "m:1")

base_final.shape

(97291, 504)

In [15]:
# dividiento variables de ingreso y gasto por mieperho, 12, ld e i00

base_final['ingreso_deflact'] = base_final['inghog1d'] / (12 * base_final['mieperho'] * base_final['ld'] * base_final['i00'])

base_final['gasto_deflact']   = base_final['gashog2d'] / (12 * base_final['mieperho'] * base_final['ld'] * base_final['i00'])

In [16]:
base_final

,aÑo,mes,nconglome,conglome,vivienda,hogar,ubigeo,dominio,estrato,periodo,...,inghog2d1,lineav_rpl,lineav,pobrezav,ubigeo_dep,dpto,aniorec,i00,ingreso_deflact,gasto_deflact
0,2019,10,007060,005001,007,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,942.402727,818.567861
1,2019,10,007060,005001,017,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,512.260548,421.077798
2,2019,10,007060,005001,028,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,NaN,NaN
3,2019,10,007060,005001,040,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,802.508596,508.609816
4,2019,10,007060,005001,050,11,010101,sierra norte,de 20 000 a 49 999 habitantes,1,...,NaN,NaN,NaN,NaN,1,1.0,2019.0,0.982802,1473.884077,1121.111288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97286,2020,12,015122,020305,052,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,11442.000000,657.0,642.814026,no vulnerable,25,25.0,2020.0,1.000000,1031.372775,963.641790
97287,2020,12,015122,020305,073,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,12083.000000,657.0,641.782898,no vulnerable,25,25.0,2020.0,1.000000,1158.018357,1043.523874
97288,2020,12,015122,020305,094,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,NaN,NaN,NaN,NaN,25,25.0,2020.0,1.000000,NaN,NaN
97289,2020,12,015122,020305,116,11,250301,selva,de 20 000 a 49 999 habitantes,2,...,21009.353516,657.0,643.160034,vulnerable no pobre,25,25.0,2020.0,1.000000,473.441602,535.855695


## Salario por hora del trabajador dependiente (Python y R)
---

In [15]:
# Cargando modulo 05
enaho05 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo05/737-Modulo05/enaho01a-2020-500.dta")
enaho05 = enaho05[ ["i524e1", "i538e1", "i513t", "i518"] ]

In [16]:
# reemplazando NA por ceros
enaho05 = enaho05.fillna(0)
enaho05

,i524e1,i538e1,i513t,i518
0,0.0,0.0,70.0,0.0
1,0.0,0.0,70.0,0.0
2,0.0,0.0,84.0,0.0
3,2433.0,0.0,12.0,70.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
91310,10761.0,0.0,53.0,0.0
91311,14202.0,0.0,48.0,0.0
91312,0.0,0.0,0.0,0.0
91313,7680.0,0.0,34.0,0.0


In [17]:
# generando variables necesarias
enaho05['ingreso'] = enaho05['i524e1'] + enaho05['i538e1']
enaho05['horas']   = enaho05['i513t'] + enaho05['i518']
enaho05['sal_hora_depend'] = enaho05['ingreso'] / (enaho05['horas'] * 52)

In [18]:
enaho05['sal_hora_depend'] = enaho05['sal_hora_depend'].replace([0], np.nan)

enaho05[['sal_hora_depend']]

,sal_hora_depend
0,NaN
1,NaN
2,NaN
3,0.570591
4,NaN
...,...
91310,3.904572
91311,5.689904
91312,NaN
91313,4.343891


## Groupby (Python y R)
---

In [19]:
enaho02 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo02/737-Modulo02/enaho01-2020-200.dta")
enaho34 = pd.read_stata("C:/Users/Jose Pastor/Documents/datos_documents/enaho/2020/737-Modulo34/737-Modulo34/sumaria-2020.dta")

In [20]:
# identificando hogares con algun miembro mayor a 65 años
enaho02_1 = enaho02.groupby( [ "conglome", "vivienda", "hogar" ],
                            as_index = False ).agg( edad_max = ( 'p208a', max ))

enaho02_1

,conglome,vivienda,hogar,edad_max
0,005002,011,11,66.0
1,005002,023,11,62.0
2,005002,049,11,74.0
3,005002,073,11,60.0
4,005002,097,11,80.0
...,...,...,...,...
34485,020353,023,11,42.0
34486,020353,030,11,32.0
34487,020353,049,11,59.0
34488,020353,055,11,47.0


In [21]:
# inner merge
base_final_2  = pd.merge(enaho02_1, enaho34,
                       on = ["conglome", "vivienda", "hogar"],
                       how = "left",
                       validate = "m:1")

In [22]:
# Generamos variable "pobre" mediante comparación entre gasto y linea de pobreza

base_final_2['gasto_month'] = base_final_2['gashog2d'] / (12*base_final_2['mieperho'])
base_final_2["pobre"] = np.where(base_final_2["gasto_month"] < base_final_2["linea"], "pobre", "no pobre")

#### Dummy mas de 65 años y pobre

In [32]:
# La estrategia es generara dummies tanto para pobreza y mayor a 65 años
# Luego se sumaran las dummies y aquellas que resulten 2 es porque cumple la condicion.
# Finalmente, se convertira el 2 en 1 para tener una dummy que cumpla con las condiciones.

base_final_2["dummy_pobre"]   = np.where(base_final_2["pobre"] == 'pobre', 1, 0)
base_final_2["dummy_mayor65"] = np.where(base_final_2["edad_max"] > 65, 1, 0)

# sumando dummies
base_final_2['dummy_suma'] = base_final_2["dummy_pobre"] + base_final_2["dummy_mayor65"]

# reemplazando 2 por 1
base_final_2['dummy_pobre_mayor65'] = np.where(base_final_2['dummy_suma'] == 2, 1, 0)


In [36]:
base_final_2.head(30)

,conglome,vivienda,hogar,edad_max,aÑo,mes,ubigeo,dominio,estrato,mieperho,...,sub_conglome,lineav_rpl,lineav,pobrezav,gasto_month,pobre,dummy_pobre,dummy_mayor65,dummy_suma,dummy_pobre_mayor65
0,005002,011,11,66.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",4,...,00,657,553.702698,no vulnerable,647.939006,no pobre,0,1,1,0
1,005002,023,11,62.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,00,657,558.874207,no vulnerable,1993.775391,no pobre,0,0,0,0
2,005002,049,11,74.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",4,...,00,657,551.961121,no vulnerable,2173.834961,no pobre,0,1,1,0
3,005002,073,11,60.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",2,...,00,657,556.134399,no vulnerable,1173.763509,no pobre,0,0,0,0
4,005002,097,11,80.0,2020,03,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,00,657,562.409973,vulnerable no pobre,328.488627,no pobre,0,1,1,0
5,005005,009,11,57.0,2020,06,010101,sierra norte,"de 20,000 a 49,999 habitantes",3,...,00,657,543.538696,no vulnerable,646.295247,no pobre,0,0,0,0
6,005005,014,11,53.0,2020,06,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,00,657,554.552917,no vulnerable,1038.844645,no pobre,0,0,0,0
7,005005,027,11,54.0,2020,06,010101,sierra norte,"de 20,000 a 49,999 habitantes",1,...,00,657,560.707520,vulnerable no pobre,505.978841,no pobre,0,0,0,0
8,005005,033,11,76.0,2020,06,010101,sierra norte,"de 20,000 a 49,999 habitantes",3,...,00,657,551.968506,no vulnerable,606.298231,no pobre,0,1,1,0
9,005005,037,11,46.0,2020,06,010101,sierra norte,"de 20,000 a 49,999 habitantes",3,...,00,657,554.594543,no vulnerable,648.639757,no pobre,0,0,0,0
